In [1]:
import torch
from tqdm.notebook import tqdm
import numpy as np
from transformer.reimei import ReiMei, ReiMeiParameters
import matplotlib.pyplot as plt
from config import AE_CHANNELS, DIT_S as DIT, MODELS_DIR_BASE, SIGLIP_HF_NAME, BERT_HF_NAME, SIGLIP_EMBED_DIM, BERT_EMBED_DIM
from torch.amp import autocast
from transformers import SiglipTokenizer, SiglipTextModel, AutoTokenizer, ModernBertModel
from diffusers import AutoencoderDC
import time

A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "d:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\xformers\__init__.py", line 57, in _is_triton_available
    import triton  # noqa
    ^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'triton'


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
device

'cuda'

In [4]:
# base_dim = 1024
# base_heads = 32

input_dim = AE_CHANNELS
num_layers = 24
embed_dim = 1152
num_heads = embed_dim // 32
mlp_dim = embed_dim
num_experts = 16
capacity_factor = 2.0
shared_experts = 1
token_mixer_layers = 2
image_text_expert_ratio = 4
dropout = 0.1

# m_d = float(embed_dim) / float(base_dim)

# assert (embed_dim // num_heads) == (base_dim // base_heads)

params = ReiMeiParameters(
    channels=input_dim,
    embed_dim=embed_dim,
    num_layers=num_layers,
    num_heads=num_heads,
    mlp_dim=mlp_dim,
    siglip_dim=SIGLIP_EMBED_DIM,
    bert_dim=BERT_EMBED_DIM,
    num_experts=num_experts,
    capacity_factor=capacity_factor,
    shared_experts=shared_experts,
    dropout=dropout,
    token_mixer_layers=token_mixer_layers,
    image_text_expert_ratio=image_text_expert_ratio,
    # m_d=m_d,
)
DTYPE = torch.bfloat16

In [5]:
model = ReiMei(params)
# model.load_state_dict(torch.load("models/reimei_model_and_optimizer_epoch_40_f32.pt")['ema_model_state_dict'])

In [6]:
# Print the number of parameters in the model
print("Number of parameters in the model: ", sum(p.numel() for p in model.parameters()))


Number of parameters in the model:  2290123320


In [7]:
siglip_model = SiglipTextModel.from_pretrained(SIGLIP_HF_NAME, cache_dir=f"{MODELS_DIR_BASE}/siglip").to(device)
siglip_tokenizer = SiglipTokenizer.from_pretrained(SIGLIP_HF_NAME, cache_dir=f"{MODELS_DIR_BASE}/siglip")

In [8]:
bert_model = ModernBertModel.from_pretrained(BERT_HF_NAME, cache_dir=f"{MODELS_DIR_BASE}/modernbert").to(device)
bert_tokenizer = AutoTokenizer.from_pretrained(BERT_HF_NAME, cache_dir=f"{MODELS_DIR_BASE}/modernbert")

In [9]:
padded_len = 64


In [10]:
prompts = [
    "A group of people in traditional Indian attire gather around a table laden with food, blending traditional and modern elements in a casual and relaxed atmosphere." 
] * 9

In [11]:
t = time.time()

siglip_inputs = []
unpadded_lens = []
for prompt in prompts:
    input = siglip_tokenizer.encode(prompt, return_tensors="pt", padding=True).to("cuda").squeeze()
    unpadded_lens.append(input.shape[-1])
    if input.shape[0] < padded_len:
        padding = torch.zeros(padded_len - input.shape[0], dtype=input.dtype, device=input.device)
        input = torch.cat([input, padding])
    else:
        input = input[:padded_len]
    siglip_inputs.append(input)

siglip_inputs = torch.stack(siglip_inputs)
siglip_outputs = siglip_model(siglip_inputs, output_hidden_states=True)
siglip_embeddings = siglip_outputs.hidden_states[-1]
siglip_vec = siglip_outputs.pooler_output
print(f"Took: {time.time() - t}")

Took: 0.3059992790222168


In [12]:
t = time.time()

bert_inputs = []
unpadded_lens = []
for prompt in prompts:
    input = bert_tokenizer.encode("[CLS]"+prompt, return_tensors="pt", padding=True).to("cuda").squeeze()
    unpadded_lens.append(input.shape[-1])
    if input.shape[0] < 65:
        padding = torch.ones(65 - input.shape[0], dtype=input.dtype, device=input.device)
        input = torch.cat([input, padding])
    else:
        input = input[:65]
    bert_inputs.append(input)

bert_inputs = torch.stack(bert_inputs)
bert_outputs = bert_model(bert_inputs, output_hidden_states=True).hidden_states[-1] # (bs, 65, 768). The 65 is CLS + 64 tokens. So we need to seperate the CLS token from the rest.
bert_vec = bert_outputs[:, 0, :] # (bs, 1024)
bert_embeddings = bert_outputs[:, 1:, :] # (bs, 64, 1024)

print(f"Took: {time.time() - t}")

Took: 0.18300104141235352


In [13]:
bert_embeddings.shape

torch.Size([9, 64, 1024])

In [14]:
siglip_embeddings.shape, 

(torch.Size([9, 64, 1152]),)

In [15]:
del siglip_model, bert_model, siglip_tokenizer, bert_tokenizer

In [16]:
noise = torch.randn(9, 32, 16, 16).to(device).to(torch.bfloat16)

In [17]:
from torch.nn.modules.normalization import RMSNorm

norm = RMSNorm(1024).to("cuda")
normed_emb = norm(bert_embeddings)
normed_vec = norm(bert_vec)

In [18]:
print("noise", torch.std_mean(noise))
print("siglip emb", torch.std_mean(siglip_embeddings))
print("siglip vec", torch.std_mean(siglip_vec))
print("bert emb", torch.std_mean(normed_emb))
print("bert vec", torch.std_mean(normed_vec))

noise (tensor(1., device='cuda:0', dtype=torch.bfloat16), tensor(-0.0057, device='cuda:0', dtype=torch.bfloat16))
siglip emb (tensor(1.1179, device='cuda:0', grad_fn=<StdMeanBackward0>), tensor(0.7216, device='cuda:0', grad_fn=<StdMeanBackward0>))
siglip vec (tensor(1.0028, device='cuda:0', grad_fn=<StdMeanBackward0>), tensor(-0.0570, device='cuda:0', grad_fn=<StdMeanBackward0>))
bert emb (tensor(1.0000, device='cuda:0', grad_fn=<StdMeanBackward0>), tensor(0.0033, device='cuda:0', grad_fn=<StdMeanBackward0>))
bert vec (tensor(0.9999, device='cuda:0', grad_fn=<StdMeanBackward0>), tensor(0.0148, device='cuda:0', grad_fn=<StdMeanBackward0>))


In [19]:
model = model.eval().to(device, dtype=torch.bfloat16)
with autocast("cuda", dtype=torch.bfloat16):
    pred = model(noise, torch.ones((9,), device="cuda"), siglip_embeddings, siglip_vec, bert_embeddings, bert_vec, None)
    # del model

In [20]:
torch.std_mean(pred)

(tensor(1.3828, device='cuda:0', dtype=torch.bfloat16,
        grad_fn=<StdMeanBackward0>),
 tensor(-0.0042, device='cuda:0', dtype=torch.bfloat16,
        grad_fn=<StdMeanBackward0>))

In [21]:
vae = AutoencoderDC.from_pretrained("mit-han-lab/dc-ae-f32c32-in-1.0-diffusers", torch_dtype=DTYPE, cache_dir=f"{MODELS_DIR_BASE}/dc_ae", revision="main").to(device, ).eval()
# print param count
print(sum(p.numel() for p in vae.parameters()))

KeyboardInterrupt: 

In [26]:
# model = model.eval()
with autocast("cuda", dtype=torch.bfloat16):
    # pred = model.sample(noise, siglip_embeddings, siglip_vec, bert_embeddings, bert_vec, sample_steps=50, cfg=7.0)

    pred = vae.decode(pred).sample
    # Change range of pred from x to y to -1 to 1
    min_val = pred.min()
    max_val = pred.max()

    pred = (pred - min_val) / (max_val - min_val)
    pred = 2 * pred - 1

In [ ]:
with torch.inference_mode():
    pred_cpu = pred.cpu().to(torch.float32)
    pred_np = pred_cpu.permute(0, 2, 3, 1).numpy()
    pred_np = (pred_np + 1) / 2
    pred_np = (pred_np * 255).astype(np.uint8)  # Convert to uint8

    # Create figure for predictions
    fig_pred, axes_pred = plt.subplots(3, 3, figsize=(12, 12))  # 3 rows, 3 columns

    # Plot predictions
    for i, ax in enumerate(axes_pred.flatten()):
        if i < pred_np.shape[0]:
            ax.imshow(pred_np[i])
            # ax.set_title(prompts[i])  # Add this line to set the title
            ax.axis('off')
        else:
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()